# Assignment 1: SARSA & Q-Learning Algorithms

In [ ]:
from math import floor
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns
from IPython.display import clear_output
import time
%matplotlib inline

import gridworld
from gridworld import *

## 1. Environment Details

In [ ]:
UP = 0
DOWN = 1
LEFT = 2
RIGHT = 3
actions = [UP, DOWN, LEFT, RIGHT]

In [ ]:
#Heatmap function
def plot_Q(Q, message = "Q plot"):
    
    plt.figure(figsize=(10,10))
    plt.title(message)
    plt.pcolor(Q.max(-1), edgecolors='k', linewidths=2)
    plt.colorbar()
    def x_direct(a):
        if a in [UP, DOWN]:
            return 0
        return 1 if a == RIGHT else -1
    def y_direct(a):
        if a in [RIGHT, LEFT]:
            return 0
        return 1 if a == UP else -1
    policy = Q.argmax(-1)
    policyx = np.vectorize(x_direct)(policy)
    policyy = np.vectorize(y_direct)(policy)
    idx = np.indices(policy.shape)
    plt.quiver(idx[1].ravel()+0.5, idx[0].ravel()+0.5, policyx.ravel(), policyy.ravel(), pivot="middle", color='red')
    plt.show()

In [ ]:
#World parameters
num_rows = 10
num_cols = 10
obstructions = np.array([[0,7],[1,1],[1,2],[1,3],[1,7],[2,1],[2,3],
                         [2,7],[3,1],[3,3],[3,5],[4,3],[4,5],[4,7],
                         [5,3],[5,7],[5,9],[6,3],[6,9],[7,1],[7,6],
                         [7,7],[7,8],[7,9],[8,1],[8,5],[8,6],[9,1]])
bad_states = np.array([[1,9],[4,2],[4,4],[7,5],[9,9]])
restart_states = np.array([[3,7],[8,2]])
start_state = np.array([[0,4]])
goal_states = np.array([[0,9],[2,2],[8,7]])

In [ ]:
#Visualising the grid
def heatmap(num_rows, num_cols, obstructions, bad_states, restart_states, start_state, goal_states):
    
    grid = np.zeros([num_rows, num_cols])
    state_list = [obstructions, bad_states, restart_states, start_state, goal_states]
    for i in range(len(state_list)):
        for r in range(state_list[i].shape[0]):
            grid[state_list[i][r][0], state_list[i][r][1]] = heat[i]
    return grid

heat = [10, 6, 8, 5, 3]
grid = heatmap(num_rows, num_cols, obstructions, bad_states, restart_states, start_state, goal_states)
sns.set()
ax = sns.heatmap(grid, vmin=0, vmax=10, cmap="Blues")
sns.color_palette("tab10")
plt.show()
print(grid)

## 2. Creating Grid World

In [ ]:
#Creating model
gw = gridworld.GridWorld(num_rows=num_rows,
               num_cols=num_cols,
               start_state=start_state,
               goal_states=goal_states, wind = False)
gw.add_obstructions(obstructed_states=obstructions,
                    bad_states=bad_states,
                    restart_states=restart_states)
gw.add_rewards(step_reward=-1,
               goal_reward=10,
               bad_state_reward=-6,
               restart_state_reward=-100)

gw.add_transition_probability(p_good_transition=1.0, bias=0.5)

env = gw.create_gridworld()

In [ ]:
print("Number of actions", env.num_actions) #0 -> UP, 1-> DOWN, 2 -> LEFT, 3-> RIGHT
print("Number of states", env.num_states)
print("start state", env.start_state_seq)
print("goal state(s)", env.goal_states_seq)

## 3. Learning Algorithms: Q Learning & SARSA

### Exploration strategies
1. Epsilon-greedy
2. Softmax

In [ ]:
#Hyperparameters
alpha0 = 0.4
gamma0 = 0.9
episodes = 10000
test_episodes = 1000
epsilon0 = 0.1
beta0 = 0.1 #[0.001, 1.0, 5.0, 50.0]
max_steps = 100

In [ ]:
from scipy.special import softmax

seed = 42
rg = np.random.RandomState(seed)

#Epsilon greedy
def choose_action_epsilon(Q, state, epsilon, rg=rg):
    if rg.rand() < epsilon:
        return rg.randint(len(actions))
    else:
        return np.argmax(Q[state[0,0]][state[0,1]])

#Softmax
def choose_action_softmax(Q, state, beta, rg=rg):
    prob = softmax(Q[state[0,0]][state[0,1]]/beta)
    best_action = rg.choice(actions, p = prob)
    return best_action

### 3.1 SARSA Algorithm
Update equation:
\begin{equation}
Q(s_t,a_t) \leftarrow  Q(s_t, a_t) + \alpha[r_t + \gamma Q(s_{t+1}, a_{t+1}) - Q(s_t, a_t)]
\end{equation}

#### Hyperparameters

We have 3 hyperparameters for the algorithm:
- $\alpha$
- number of *episodes*.
- $\epsilon$: For epsilon greedy exploration

In [ ]:
print_freq = 100

def sarsa(env, Q, gamma, alpha, policy_parameter, episodes, plot_heat, choose_action):

    episode_rewards = np.zeros(episodes)
    steps_to_completion = np.zeros(episodes)
    if plot_heat:
        clear_output(wait=True)
        plot_Q(Q)

    state_visits = np.zeros((num_rows, num_cols))
    for ep in range(episodes):
        tot_reward, steps = 0, 0
        
        #Reset environment
        state = env.reset()
        action = choose_action(Q, seq_to_col_row(state, num_cols), policy_parameter)
        # done = False
        while steps<max_steps:
            state_next, reward = env.step(state,action)
            state_rc = seq_to_col_row(state, num_cols)
            state_next_rc = seq_to_col_row(state_next, num_cols)
            state_visits[state_rc[:,0], state_rc[:,1]] += 1

            #print(state_next,gw.goal_states_seq)
            if any(np.sum(np.abs(gw.goal_states-state_rc),1)==0):
                break
            action_next = choose_action(Q, seq_to_col_row(state_next, num_cols), policy_parameter)
            
            Q[state_rc[:,0], state_rc[:,1], actions.index(action)] = Q[state_rc[:,0], state_rc[:,1], actions.index(action)] + alpha*(reward + gamma*Q[state_next_rc[:,0], state_next_rc[:,1], actions.index(action_next)] - Q[state_rc[:,0], state_rc[:,1], actions.index(action)])
                                                    
            tot_reward += reward
            steps += 1
            
            state, action = state_next, action_next
        
        episode_rewards[ep] = tot_reward
        steps_to_completion[ep] = steps

        if (ep+1)%print_freq == 0 and plot_heat:
            clear_output(wait=True)
            plot_Q(Q, message = f"alpha={alpha},gamma={gamma},epsilon={policy_parameter} "+"Episode %d: Reward: %f, Steps: %.2f, Qmax: %.2f, Qmin: %.2f"%(ep+1, np.mean(episode_rewards[ep-print_freq+1:ep]),
                                                                           np.mean(steps_to_completion[ep-print_freq+1:ep]),
                                                                           Q.max(), Q.min()))
                
    return Q, episode_rewards, steps_to_completion, state_visits

### 3.2 Q-Learning Algorithm
Update rule for Q-Learning:
\begin{equation}
Q(s_t,a_t) \leftarrow  Q(s_t, a_t) + \alpha[r_t + \gamma \max_a Q(s_{t+1}, a) - Q(s_t, a_t)]
\end{equation}

In [ ]:
print_freq = 100

def qlearning(env, Q, gamma, alpha, policy_parameter, plot_heat = False, choose_action = choose_action_softmax):

    episode_rewards = np.zeros(episodes)
    steps_to_completion = np.zeros(episodes)
    
    if plot_heat:
        clear_output(wait=True)
        plot_Q(Q)
    
    for ep in range(episodes):
        tot_reward, steps = 0, 0
        
        #Reset environment

        state = env.reset()
        action = choose_action(Q, seq_to_col_row(state, num_cols), policy_parameter)

        done = False
        while not done:
            state_next, reward, done = env.step(action)
            action_next = choose_action(Q, state_next)
            
            Q[state[0], state[1], actions.index(action)] = Q[state[0], state[1], actions.index(action)] + alpha*(reward + gamma*np.max(Q[state_next[0], state_next[1], :]) - Q[state[0], state[1], actions.index(action)])
                                                    
            tot_reward += reward
            steps += 1
            
            state, action = state_next, action_next
        
        episode_rewards[ep] = tot_reward
        steps_to_completion[ep] = steps
        
        if (ep+1)%print_freq == 0 and plot_heat:
            clear_output(wait=True)
            plot_Q(Q, message = "Episode %d: Reward: %f, Steps: %.2f, Qmax: %.2f, Qmin: %.2f"%(ep+1, np.mean(episode_rewards[ep-print_freq+1:ep]),
                                                                           np.mean(steps_to_completion[ep-print_freq+1:ep]),
                                                                           Q.max(), Q.min()))
                
    return Q, episode_rewards, steps_to_completion

# 4. Hyperparameter Search
Performing search for optimum set hyperparameters for each of the 16 configurations for each algorithm.

In [ ]:
#Initializing test hyperparameters 
test_steps = []
test_rewards = []

#Choosing values both emperically and after some test runs
test_alpha_list = [0.001, 0.4, 0.9]
test_beta_list = [0.1, 1.0, 5.0]
test_epsilon_list = [0.0001, 0.2]
test_gamma_list = [0, 0.4, .9]

test_parameters = [test_alpha_list, test_beta_list, test_epsilon_list, test_gamma_list]
parameter_names = ['Alpha', 'Beta', 'Epsilon', 'Gamma']

In [ ]:
def optimum_combination_steps(test_array, test_policy_parameter):
    '''
    Returns
    '''
    indices = np.where(test_array == np.min(test_array))
    return test_alpha_list[indices[0][0]], test_policy_parameter[indices[1][0]], test_gamma_list[indices[2][0]]

def hyperparam_search(algorithm, plot_heat):
    
    #Initializing matrices for optimum params
    optimum_alpha_ep = np.zeros((2,2,2))
    optimum_alpha_soft = np.zeros((2,2,2))

    optimum_epsilon_ep = np.zeros((2,2,2))

    optimum_beta_soft = np.zeros((2,2,2))

    optimum_gamma_ep = np.zeros((2,2,2))
    optimum_gamma_soft = np.zeros((2,2,2))

    configuration = 1

    #Looping over all configurations
    for wind in [False, True]:
        for start_state in [np.array([[0,4]]), np.array([[3,6]])]:
            for p_good_transition in [1.0, 0.7]:
            
                gw = gridworld.GridWorld(num_rows=num_rows, num_cols=num_cols, start_state=start_state, goal_states=goal_states, wind=wind)
                gw.add_obstructions(obstructed_states=obstructions, bad_states=bad_states, restart_states=restart_states)
                gw.add_rewards(step_reward=-1, goal_reward=10, bad_state_reward=-6, restart_state_reward=-100)
                gw.add_transition_probability(p_good_transition=p_good_transition, bias=0.5)
                env = gw.create_gridworld()
                
                if wind == False:
                    i = 0
                else: 
                    i = 1

                if (start_state == np.array([[0,4]])).all():
                    j = 0
                else:
                    j = 1
                
                if p_good_transition == 1.0:
                    k = 0
                else:
                    k = 1

                test_steps_ep = np.zeros((3, 2, 3))
                test_rewards_ep = np.zeros((3, 2, 3))

                print(f"Searching for configuration: {configuration}", "\r")
                
                start = time.time()
                for alpha in test_parameters[0]:
                    for epsilon in test_parameters[2]:
                        for gamma in test_parameters[3]:
                            Q = np.zeros((env.num_rows, env.num_cols, len(actions)))    
                            Q, rewards, steps, state_visits = algorithm(env, Q, gamma, alpha, epsilon, test_episodes, plot_heat, choose_action = choose_action_epsilon)
                            test_steps_ep[test_parameters[0].index(alpha), test_parameters[2].index(epsilon), test_parameters[3].index(gamma)] = np.mean(steps)
                            test_rewards_ep[test_parameters[0].index(alpha), test_parameters[2].index(epsilon), test_parameters[3].index(gamma)] = np.mean(rewards)

                test_steps_soft = np.zeros((3, 3, 3))
                test_rewards_soft = np.zeros((3, 3, 3))
                
                for alpha in test_parameters[0]:
                    for beta in test_parameters[1]:
                        for gamma in test_parameters[3]:
                            Q = np.zeros((env.num_rows, env.num_cols, len(actions)))    
                            Q, rewards, steps, state_visits = algorithm(env, Q, gamma, alpha, beta, test_episodes, plot_heat, choose_action = choose_action_softmax)
                            test_steps_soft[test_parameters[0].index(alpha), test_parameters[1].index(beta), test_parameters[3].index(gamma)] = np.mean(steps)
                            test_rewards_soft[test_parameters[0].index(alpha), test_parameters[1].index(beta), test_parameters[3].index(gamma)] = np.mean(rewards)

                end = time.time()

                a, e, g = optimum_combination_steps(test_steps_ep, test_epsilon_list)
                optimum_alpha_ep[i, j, k] = a
                optimum_epsilon_ep[i, j, k] = e
                optimum_gamma_ep[i, j, k] = g
                # optimum_combination_rewards(test_rewards_soft, test_epsilon_list)
                a, b, g = optimum_combination_steps(test_steps_soft, test_beta_list)
                # optimum_combination_rewards(test_rewards_soft, test_beta_list)
                optimum_alpha_soft[i, j, k] = a
                optimum_beta_soft[i, j, k] = b
                optimum_gamma_soft[i, j, k] = g

                print(f"Time of execution for configuration {configuration} = {end-start}s")
                configuration += 1

    return [[optimum_alpha_ep, optimum_epsilon_ep, optimum_gamma_ep], [optimum_alpha_soft, optimum_beta_soft, optimum_gamma_soft]]  

Control the *plot_heat* function as you like!

In [ ]:
optimum_sarsa = hyperparam_search(sarsa,plot_heat=False)
optimum_qlearning = hyperparam_search(qlearning, plot_heat=False)

In [ ]:
# print(optimum_alpha_soft)
# print('\n---------\n')
# print(optimum_beta_soft)
# print('\n---------\n')
# print(optimum_gamma_soft)

In [ ]:
# num_expts = 5
# reward_avgs, steps_avgs = [], []

# for i in range(num_expts):
#     print("Experiment: %d"%(i+1))
#     Q = np.zeros((env.num_rows, env.num_cols, len(actions)))
#     rg = np.random.RandomState(i)

#     Q, rewards, steps = sarsa(env, Q, gamma = gamma, plot_heat=True, choose_action= choose_action_softmax)
#     steps_avgs.append(steps)
#     reward_avgs.append(rewards)

# steps_avgs = np.sum(steps_avgs,axis=0)/num_expts
# reward_avgs = np.sum(reward_avgs,axis=0)/num_expts

In [ ]:
# plt.figure()
# plt.plot(range(1000),steps_avgs)
# plt.xlabel('Episode')
# plt.ylabel('Number of steps to Goal')
# plt.show()

# plt.figure()
# plt.plot(range(1000),reward_avgs)
# plt.xlabel('Episode')
# plt.ylabel('Total Reward')
# plt.show()